In [1]:
import cv2
import pandas as pd
from copy import deepcopy
from pathlib import Path
import cv2
import matplotlib.pylab as plt
import numpy as np
from scipy.io import loadmat
from scipy.ndimage import gaussian_filter
from tqdm import tqdm
import cv2
import pandas as pd
import numpy as np 
import cv2 
from matplotlib import pyplot as plt 
import math
import os
import cv2
from scipy.io import loadmat
from copy import deepcopy
from PIL import Image
# csv_path = '129_2_clean.txt'
# df1 = pd.read_csv(csv_path)

In [ ]:
df1

In [ ]:
# img = Image.open("frame_800_without_boxes_right.jpg")
# width, height = img.size
# print(width,height)

In [ ]:
# output_folder = "labels_yolov8"
# os.makedirs(output_folder, exist_ok=True)

In [ ]:
# # Write each row to a separate text file
# for index, row in df1.iterrows():
#     file_path = os.path.join(output_folder, f'{int(row["frame"])}.txt')
#     with open(file_path, 'a') as f:
#         # Calculate normalized box width and height
#         image_width = 1920  # Replace with your actual image width
#         image_height = 1080  # Replace with your actual image height
#         x, y, x_offset, y_offset, obj_id = float(row['x']), float(row['y']), float(row['x_offset']), float(row['y_offset']), float(row['id'])
#         xmin, ymin, xmax, ymax = x, y, x + x_offset, y + y_offset
#         width = x_offset / image_width
#         height = y_offset / image_height
#         x_center = (x + x_offset / 2) / image_width
#         y_center = (y + y_offset / 2) / image_height
        
#         # Write data in the required format separated by spaces
#         f.write(f'1 {x_center} {y_center} {width} {height}\n')

In [ ]:
# import os

# # Specify the folder path
# folder_path = 'datasets/val/images'

# # Get all the files in the folder
# files = os.listdir(folder_path)

# # Loop through each file
# for index, file_name in enumerate(files):
#     # Construct the new file name
#     if file_name != '.DS_Store':
#         print(file_name)
#         file_num = int(file_name.split('_frame_')[1].split('_undistorted.jpg')[0])
#         new_file_name = f'{file_num}.jpg'  # Change the extension as per your file type
#         print(new_file_name)
#     #     Construct the full paths of the old and new files
#         old_file_path = os.path.join(folder_path, file_name)
#         new_file_path = os.path.join(folder_path, new_file_name)

#         # Rename the file
#         os.rename(old_file_path, new_file_path)

In [11]:
import cv2
import pandas as pd
import os

def prepare_data_for_yolov8(video_path, csv_path, output_dir):
    """
    Prepares data from a video file and CSV for YOLOv8 training or inference.

    Args:
        video_path (str): Path to the video file.
        csv_path (str): Path to the CSV file containing object coordinates.
        output_dir (str): The output directory where images and labels will be saved.
    """

    # Create output directories for images and labels
    images_dir = os.path.join(output_dir, 'images')
    labels_dir = os.path.join(output_dir, 'labels')
    os.makedirs(images_dir, exist_ok=True)
    os.makedirs(labels_dir, exist_ok=True)

    # Load the CSV data
    data = pd.read_csv(csv_path)

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    frame_index = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Save the image frame
        image_path = os.path.join(images_dir, f'frame_{frame_index}.jpg')
        cv2.imwrite(image_path, frame)

        # Create the corresponding label file
        label_path = os.path.join(labels_dir, f'frame_{frame_index}.txt')
        frame_data = data[data['frame'] == frame_index]

        with open(label_path, 'w') as f:
            for _, row in frame_data.iterrows():
                obj_id = row['id']  # Assuming you have an 'id' column
                x = row['x']
                y = row['y']
                x_offset = row['x_offset']
                y_offset = row['y_offset']
                image_width = frame.shape[1]  
                image_height = frame.shape[0] 

                # Calculate bounding box coordinates
                top_left_x = x - x_offset
                top_left_y = y - y_offset
                width = 2 * x_offset  # Assuming offset is half the width
                height = 2 * y_offset  # Assuming offset is half the height

                # Normalize coordinates
                center_x = (top_left_x + width / 2) / image_width
                center_y = (top_left_y + height / 2) / image_height
                width = width / image_width
                height = height / image_height

                # Assume class label is always 0 (you may need to change this) 
                class_label = 0  

                f.write(f"{class_label} {center_x} {center_y} {width} {height}\n")

        frame_index += 1

    cap.release()

# Example usage
video_path = '../vids/349_1.mp4'
csv_path = '349_1_clean.txt'
output_dir = 'dataset_349_1' 
prepare_data_for_yolov8(video_path, csv_path, output_dir)


In [24]:
import cv2
import pandas as pd
import os
import argparse

def prepare_data_for_yolov8(video_paths, csv_paths, output_dir):
    """
    Prepares data from multiple video files and CSVs for YOLOv8 training or inference.

    Args:
        video_paths (list): List of paths to the video files.
        csv_paths (list): List of paths to the CSV files containing object coordinates.
        output_dir (str): The output directory where images and labels will be saved.
    """

    # Input validation
    if len(video_paths) != len(csv_paths):
        raise ValueError("The number of video paths and CSV paths must match.")

    # Create output directories
    images_dir = os.path.join(output_dir, 'images')
    labels_dir = os.path.join(output_dir, 'labels')
    os.makedirs(images_dir, exist_ok=True)
    os.makedirs(labels_dir, exist_ok=True)

    # Process each video and corresponding CSV
    for video_path, csv_path in zip(video_paths, csv_paths):
        base_filename = os.path.splitext(os.path.basename(video_path))[0] 

        # Load the CSV data
        data = pd.read_csv(csv_path)

        # Open the video file
        cap = cv2.VideoCapture(video_path)

        frame_index = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Save the image 
            image_path = os.path.join(images_dir, f'{base_filename}_{frame_index}.jpg')
            cv2.imwrite(image_path, frame)

            # Create the label file
            label_path = os.path.join(labels_dir, f'{base_filename}_{frame_index}.txt')
            frame_data = data[data['frame'] == frame_index]

            with open(label_path, 'w') as f:
                for _, row in frame_data.iterrows():
                    obj_id = row['id']  
                    x = row['x']
                    y = row['y']
                    x_offset = row['x_offset']
                    y_offset = row['y_offset']
                    image_width = frame.shape[1]  
                    image_height = frame.shape[0]  

                    # Calculate bounding box coordinates
                    top_left_x = x - x_offset
                    top_left_y = y - y_offset
                    width = 2 * x_offset  
                    height = 2 * y_offset  

                    # Normalize coordinates
                    center_x = (top_left_x + width / 2) / image_width
                    center_y = (top_left_y + height / 2) / image_height
                    width = width / image_width
                    height = height / image_width

                    # Assume class label is always 0 (you may need to change this) 
                    class_label = 0  

                    f.write(f"{class_label} {center_x} {center_y} {width} {height}\n")

            frame_index += 1

        cap.release()

# Example usage
video_paths = ['../vids/349_2.mp4','../vids/406_1.mp4', '../vids/161_2.mp4']
csv_paths = ['349_2_clean.txt','406_1_clean.txt','161_2_clean.txt']
output_dir = 'dataset_V2_Val' 
prepare_data_for_yolov8(video_paths, csv_paths, output_dir)


# val  ['349_2_clean.txt','406_1_clean.txt','161_2_clean.txt']


In [22]:
# l = ['../vids/161_1.mp4','../vids/183_1.mp4' ,'../vids/183_2.mp4','../vids/231_1.mp4','../vids/231_2.mp4','../vids/261_1.mp4','../vids/349_1.mp4','../vids/406_2.mp4']
# ll = ['161_1_clean.txt', '183_1_clean.txt','183_2_clean.txt','231_1_clean.txt','231_2_clean.txt','261_1_clean.txt','349_1_clean.txt','406_2_clean.txt']
# print(len(video_paths))
# print(len(csv_paths))